<a href="https://colab.research.google.com/github/devac-yabuki/LLM-Learn/blob/main/llm-introduction/notebooks/chapter01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 第1章 はじめに
## 1.1 transformers を使って自然言語処理を解いてみよう
- transformers をインストール
    - 大規模言語モデルをプログラムから扱うための標準的なライブラリ
    - オプション
        - ja
            - 日本語を扱う
        - sentencepiece
            - 分単位のバイト対符号化に使用
        - torch
            - Pytorch に関連するライブラリ

In [ ]:
!pip install transformers[ja,sentencepiece,torch]

- モデルを簡単に扱うための pipelines を使用

In [ ]:
from transformers import pipeline

### 1.1.1 文書分類
- テキストをあらかじめ定められたラベルに分解するタスク
- テキストから読み取れる感情を検出する文書分類を **感情分析** と呼ぶ


- 感情分析モデルを実行

In [ ]:
text_classification_pipeline = pipeline(
    model="llm-book/bert-base-japanese-v3-marc_ja"
)
positive_text = "世界には言葉がわからなくても感動する音楽がある。"

# positive_text の極性を予測
print(text_classification_pipeline(positive_text)[0])

In [ ]:
# negative_text の極性を予測
negative_text = "世界には言葉が出ないほどひどい音楽がある"
print(text_classification_pipeline(negative_text)[0])

### 1.1.2 自然言語推論
- 2つのテキストの論理関係を予測するタスク
- 言語モデルの意味理解能力を評価するために利用される

In [ ]:
nli_pipeline = pipeline(model="llm-book/bert-base-japanese-v3-jnli")
text = "二人の男性がジェット機を見ています"
entailment_text = "ジェット機を見ている人が二人います"

# text と entailment_text の論理関係を予測
print(nli_pipeline({"text": text, "text_pair": entailment_text}))

In [ ]:
contradiction_text = "二人の男性が飛んでいます"
# text と contradiction_text の論理関係を予測
print(nli_pipeline({"text": text, "text_pair": contradiction_text}))

In [ ]:
neutral_text = "2人の男性が、白い飛行機を眺めています"
# text と neutral_text の論理関係を予測
print(nli_pipeline({"text": text, "text_pair": neutral_text}))

- label の意味
    - entailment
        - 含意
    - contradiction
        - 矛盾
    - neutral
        - 中立 (含意とも矛盾とも判断がつかない)

### 1.1.3 意味的類似度計算
- 2つのテキストの意味が似ている度合いをスコアとして予測するタスク
- 情報検索や複数テキストの内容の整合性を確認する際に役立つ


- 与えられた2つのテキストの意味的類似度を0から5の範囲で予測する


In [ ]:
text_sim_pipeline = pipeline(
    model="llm-book/bert-base-japanese-v3-jsts",
    function_to_apply="none",
)

text = "川べりでサーフボードを持った人たちがいます"
sim_text = "サーファーたちが川べりに立っています"

# text と sim_text の類似度を計算
result = text_sim_pipeline({"text": text, "text_pair": sim_text})
print(result["score"])

In [ ]:
dissim_text = "トイレの壁に黒いタオルがかけられています"
# text と dissim_text の類似度を計算
result = text_sim_pipeline({"text": text, "text_pair": dissim_text})
print(result["score"])

- 内容が似ていれば5に近くなり、関係ない場合は0に近くなる


- 文埋め込みモデル
    - テキストのbげクトルのコサイン類似度を意味的類似度をみなすことができる

In [ ]:
from torch.nn.functional import cosine_similarity

sim_enc_pipeline = pipeline(
    model="llm-book/bert-base-japanese-v3-unsup-simcse-jawiki",
    task="feature-extraction",
)

# text と sim_text のベクトルを獲得
text_emb = sim_enc_pipeline(text, return_tensors=True)[0][0]
sim_emb = sim_enc_pipeline(sim_text, return_tensors=True)[0][0]

# text と sim_text の類似度を計算
sim_pair_score = cosine_similarity(text_emb, sim_emb, dim=0)
print(sim_pair_score.item())

In [ ]:
# dissim_text のベクトルを獲得
dissim_emb = sim_enc_pipeline(dissim_text, return_tensors=True)[0][0]

# text と dissim_text の類似度を計算
dissim_pair_score = cosine_similarity(text_emb, dissim_emb, dim=0)
print(dissim_pair_score.item())

- コサイン類似度の値の範囲は -1から1
    - 内容が似ている場合は1に近くなり、関係ない場合は-1に近くなる